Naloga pajka:
1. HTTP downloader and renderer: To retrieve and render a web page.
2. Data extractor: Minimal functionalities to extract images and hyperlinks.
3. Duplicate detector: To detect already parsed pages.
4. URL frontier: A list of URLs waiting to be parsed.
5. Datastore: To store the data and additional metadata used by the crawler.

TO-DO 2-images exctraction, duplicate detector 

In [ ]:
import concurrent.futures
import threading
import psycopg2

lock = threading.Lock()


  

def update_site_locking(domain, sitemap, robotstxt):
    conn = psycopg2.connect(host="http://83.212.127.54/", user="test", password="fricrawl")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO site (domain, robots_content,sitemap_content) VALUES (%s, %s, %s) RETURNING id;",
        (domain, sitemap, robotstxt))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
def update_page_locking(siteId, url,html_content, status_code, acc_time, page_type_code):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO page (site_id, url,html_content,http_status_code,accessed_time, page_type_code) VALUES (%s,%s,%s,%s,%s,%s) RETURNING id;",
        (siteId, url, html_content,status_code,acc_time,page_type_code))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
def update_image_locking(pageId, filename, content_type, data, acc_time):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO image (page_id,filename, content_type,data,accessed_time) VALUES (%s,%s,%s, %s, %s) RETURNING id;",
        (pageId, filename, content_type,data,acc_time))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
def update_page_data_locking(pageId, data_type_code, data):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO page_data (page_id,data_type_code, data) VALUES (%s, %s, %s) RETURNING id;",
        (pageId, data_type_code, data))
        id = cur.fetchone()[0]

    conn.commit()
    cur.close()
    conn.close()
                    
    return id;
                    
                                        
def update_link_locking(from_page, to_page):
    conn = psycopg2.connect(host="localhost", user="user", password="SecretPassword")
    conn.autocommit = True
    
    with lock:
        cur = conn.cursor()
        cur.execute("INSERT INTO link (from_page,to_page) VALUES (%s, %s);",
        (from_page, to_page))

    conn.commit()
    cur.close()
    conn.close()
                    



In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import queue
import re
from urllib.parse import urljoin
from urllib.parse import urlparse


frontier = queue.Queue()
WEB_DRIVER_LOCATION = "C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe"
TIMEOUT = 5
url =  "https://www.gov.si/"

def is_absolute(link):
    return bool(urlparse(link).netloc)

def is_link(link):
    if (len(re.findall('.:\/\/.',link))>0):
        return True
    return False
    #logika - preveri če je stvar valid link ki ga damo v frontier
    
def make_absolute(baselink,link):
    if (True != is_absolute(link)):
        return(urljoin(baselink,link))
    elif(link[0] == '/' and link[1] == '/'):
        return 'https:' + link
    return link
    #logika - prepozna če je link relativen inga spremeni v absolutnega

def get_robotstxt(link):
    chrome_options = Options()
    chrome_options.add_argument("user-agent=fri-ieps-TEST")
    driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
    driver.get(link + '/robots.txt')
    time.sleep(TIMEOUT)
    html = driver.page_source
    bsObj = BeautifulSoup(html, 'html.parser')
    robots = bsObj.find('pre').contents
    return robots

def get_sitemap(robotstxt):
    robotarray = robotstxt.split()
    print("dalje:: \n "+ robotstxt)
    if("Sitemap:" in robotarray):
        index = robotarray.index("Sitemap:")
        return robotarray[index+1]
    return None

def getsitemapContext(url):
   site = url.find_all("sitemap")
    
def crawl_site(url):

    WEB_PAGE_ADDRESS = url
    baseurl = re.match('^.+?[^\/:](?=[?\/]|$)', url).group(0)
    print(baseurl)
    
    chrome_options = Options()
    chrome_options.add_argument("user-agent=fri-ieps-TEST")
    driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
    driver.get(WEB_PAGE_ADDRESS)
    time.sleep(TIMEOUT)

    html = driver.page_source

    bsObj = BeautifulSoup(html, 'html.parser')
    links = bsObj.find_all('a', href=True)
    #finalLinks = set()
    print("tu")

    for link in links:
        if (len(link.attrs['href'])>0 and link.attrs['href'][0] != '#'):
            frontier.put(make_absolute(baseurl,link.attrs['href']))
            
    #print(list(frontier.queue))
    print("done1")
    domain = baseurl
    robotstxt = get_robotstxt(url)

    str1 = ''.join(robotstxt)

    print(str1)
    sitemaplink = get_sitemap(str1)
    sitemap = get_sitemaps(sitemaplink)
    print(sitemap)
    update_site_locking(domain,robotstxt,sitemap)

crawl_site(url)
    

https://www.gov.si


V spodnjem delu kode so metode za pridobivanja sitemapov, potrebno je dobiti .xml naslov iz trenutnega linka - kliči get_sitemap(robots.tx), ki iz robots txt pridobi sitemap.xml, spodnje  funkcije se sprehodijo
1. poženi urlSitemap=get_site_map(robots.txt)
2. get_all_urls_siteMap(urlSitemap)

Moj naslov: WEB_DRIVER_LOCATION = "C:/Work/Magisterij_1_leto/2.semester/ekstrakcijaSplet/Nal1/chromedriver.exe"
Juretov naslov: C:/Users/Pirk/Desktop/faks-mag/ekstrakcija/chromedriver.exe

In [ ]:
import pandas as pd
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import xmltodict

def get_sitemaps(url): #find all sitemaps of sitemap from robot.txt
    """Scrapes an XML sitemap from the provided URL and returns XML source.
    Args:
        url (string): Fully qualified URL pointing to XML sitemap.
    Returns:
        xml (string): XML source of scraped sitemap.
    """
    response = urllib.request.urlopen(url)
    xml = BeautifulSoup(response, 
                         'lxml-xml', 
                         from_encoding=response.info().get_param('charset'))
    return xml

def get_sitemap_type(xml):
    """Parse XML source and returns the type of sitemap.
    Args:
        xml (string): Source code of XML sitemap.
    Returns:
        sitemap_type (string): Type of sitemap (sitemap, sitemapindex, or None).
    """
    sitemapindex = xml.find_all('sitemapindex')
    sitemap = xml.find_all('urlset')
    #print(sitemap)
    if sitemapindex:
        return 'sitemapindex' #vsebujejo linke na otroke
    elif sitemap:
        return 'urlset' #direktni linki
    else:
        return
    
def get_child_sitemaps(xml):
    """Return a list of child sitemaps present in a XML sitemap file.
    Args:
        xml (string): XML source of sitemap. 
    Returns:
        sitemaps (list): Python list of XML sitemap URLs.
    """
    sitemaps = xml.find_all("sitemap")
    output = []
    for sitemap in sitemaps:
        output.append(sitemap.findNext("loc").text)
  
    return output
def sitemap_to_dataframe(xml, name=None, data=None, verbose=False):
    """Read an XML sitemap into a Pandas dataframe. 

    Args:
        xml (string): XML source of sitemap. 
        name (optional): Optional name for sitemap parsed.
        verbose (boolean, optional): Set to True to monitor progress.

    Returns:
        dataframe: Pandas dataframe of XML sitemap content. 
    """

    df = pd.DataFrame(columns=['loc', 'changefreq', 'priority', 'domain', 'sitemap_name'])

    urls = xml.find_all("url")
  
    for url in urls:

        if xml.find("loc"):
            loc = url.findNext("loc").text
            parsed_uri = urlparse(loc)
            domain = '{uri.netloc}'.format(uri=parsed_uri)
        else:
            loc = ''
            domain = ''

        if xml.find("changefreq"):
            changefreq = url.findNext("changefreq").text
        else:
            changefreq = ''

        if xml.find("priority"):
            priority = url.findNext("priority").text
        else:
            priority = ''

        if name:
            sitemap_name = name
        else:
            sitemap_name = ''
              
        row = {
            'domain': domain,
            'loc': loc,
            'changefreq': changefreq,
            'priority': priority,
            'sitemap_name': sitemap_name,
        }

        if verbose:
            print(row)

        df = df.append(row, ignore_index=True)
    return df
def get_all_urls_SiteMap(url): #provide xml of a site , from robot.txt ,... 
    """Return a dataframe containing all of the URLs from a site's XML sitemaps.
    Args:
        url (string): URL of site's XML sitemap. Usually located at /sitemap.xml
    Returns:
        df (dataframe): Pandas dataframe containing all sitemap content. 

    """  
    xml = get_sitemaps(url)
    sitemap_type = get_sitemap_type(xml)
    if sitemap_type =='sitemapindex':
        sitemaps = get_child_sitemaps(xml)
    else:
        sitemaps = [url]
    df = pd.DataFrame(columns=['loc', 'changefreq', 'priority', 'domain', 'sitemap_name'])
    for sitemap in sitemaps:
        #print(sitemap)
        sitemap_xml = get_sitemaps(sitemap) # ce želimo imeti vse povezave sitemapov
        #  sitemaps_all.append(sitemap_xml) 
        df_sitemap = sitemap_to_dataframe(sitemap_xml, name=sitemap)
        print(sitemap_xml)
        #print(df_sitemap)
        #print("end")
        df = pd.concat([df, df_sitemap], ignore_index=True)
        print(sitemap)
        #file = urllib.request.urlopen(sitemap)  #odpri xml in pridobi podatke
        #data = file.read()
        #file.close()
        #data = xmltodict.parse(data)
        #print(data)
    return df

url="https://www.gov.si/sitemap.xml" #dobimo ga recimo z robots.txt
dataFrame = get_all_urls_SiteMap(url)
#print(dataFrame.head())
#print(dataFrame.sitemap_name.value_counts())
#xml=get_sitemap(url)
#sitemap_type = get_sitemap_type(xml)
#if(sitemap_type=="sitemapindex"): #če je 
    #child_sitemaps = get_child_sitemaps(xml)
    #print(child_sitemaps)
